In [2]:
from pycalphad import Database, equilibrium, variables as v
# from pycalphad.core.utils import filter_phases
import random
import numpy as np
import time
from tqdm import tqdm
# from tqdm.auto import tqdm

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
# TODO: check 'PM' param
all_components = ['LI', 'BE', 'NA', 'MG', 'AL', 'K', 'CA', 'SC', 'TI', 'V', 'CR', 'MN', 'FE',
                  'CO', 'NI', 'CU', 'ZN', 'GA', 'Y', 'ZR', 'NB', 'MO', 'TC',
                  'RH', 'PD', 'AG', 'CD', 'IN', 'SN', 'BA', 'LA', 'CE', 'PR', 'ND', 'SM', 'EU',
                  'GD', 'TB', 'DY', 'HO', 'ER', 'HF', 'TA', 'W', 'RE', 'OS', 'IR',
                  'PT', 'AU', 'HG', 'TL', 'PB', 'BI', 'C', 'SI']
all_phases = list(db.phases)
all_phases.sort()
all_components.sort()

In [3]:
from numpy import save

save('./DataSets/all_phases.npy', all_phases)

In [15]:
import os
from modules.GetGm import getGM
from modules.Calculate import calculate



def getTimeGM(db, Components, all_phases, possible_phases, amounts,T):
    tic = time.perf_counter()
    gm = getGM(db, Components, all_phases, possible_phases, amounts,T)
    toc = time.perf_counter()
    return (toc-tic, gm)

def getPhases(int_mask, all_phases=all_phases):
    return [all_phases[i] for i in range(len(all_phases)) if (int(int_mask) & (1 << i)) > 0]

def phasesToMask(phases, all_phases=all_phases):
    phases.sort()
    mask = [1 if phase in phases else 0 for phase in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)


def phaseToMask(phase, all_phases=all_phases):
    mask = [1 if phase == phase_ else 0 for phase_ in all_phases][::-1]
    return int(''.join(map(str, mask)), 2)

def compToMask(components, all_components=all_components):
    components.sort()
    mask = [1 if comp in components else 0 for comp in all_components][::-1]
    return int(''.join(map(str, mask)), 2)

def getComponents(int_mask, all_components=all_components):
    return [all_components[i] for i in range(len(all_components)) if (int_mask & (1 << i)) > 0]

def parseArray(amounts_str):
    amounts = amounts_str.replace('[', '').replace(']', '')
    return np.array(amounts.split(','), dtype=np.float32)

def addDFLineToCSV(df, path='./DataSets/parsedDataset.csv'):
    df.to_csv(path, mode='a', sep=";", header=not os.path.exists(path))


In [16]:
import pandas as pd

df_origin = pd.read_csv('DataSets/main_test.csv',delimiter = ';')
df_origin = df_origin[df_origin['Error'].isna()].reset_index(drop=True)
df = df_origin.reset_index(drop=True)
df_origin['gm_time'] = np.nan
df_origin['GM'] = np.nan
df_origin['GM'] = df_origin['GM'].astype(object)
df = df.drop(['Error'], axis =1)
df = df.rename(columns={"T": "Temp"})
df['Components'] = df['Components'].apply(getComponents)
df['possible_phases'] = df['possible_phases'].apply(getPhases)
df['phases'] = df['phases'].apply(getPhases)
df['amounts'] = df['amounts'].apply(parseArray)
# print(df_origin.dtypes)
# print(df.dtypes)
df

,iter,Temp,amounts,Components,phases,ellapsed_time,possible_phases,P
0,500,1991,"[0.106400184, 0.070539184, 0.09098721, 0.08310...","[CO, CR, MO, NI, TA, V, W, ZR]",[LIQUID],28.562051,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
1,501,1613,"[0.055818617, 0.064889625, 0.21049123, 0.10578...","[CO, CR, MO, NI, TA, V, W, ZR]",[],25.476817,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
2,502,1253,"[0.0770591, 0.054417454, 0.11515643, 0.1760729...","[CO, CR, MO, NI, TA, V, W, ZR]","[LAVES_C14, LAVES_C15, NIZR, PT2V]",25.641211,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
3,503,429,"[0.1490746, 0.3030639, 0.28587595, 0.050671652...","[CO, CR, MO, NI, TA, V, W, ZR]","[CO2ZR, COV3_A15, LAVES_C14, LAVES_C15, MU_PHA...",24.184063,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
4,504,1324,"[0.29517522, 0.29847562, 0.06297051, 0.1217284...","[CO, CR, MO, NI, TA, V, W, ZR]","[LAVES_C14, LIQUID, MU_PHASE, SIGMA]",24.452664,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
...,...,...,...,...,...,...,...,...
735,1233,993,"[0.3217174, 0.22605425, 0.19374935, 0.05437803...","[CO, CR, MO, NI, TA, V, W, ZR]","[ALTA_SIGMA, CO2ZR, LAVES_C14, LAVES_C15, MU_P...",21.828628,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
736,1234,566,"[0.10788198, 0.27838793, 0.09425418, 0.1218519...","[CO, CR, MO, NI, TA, V, W, ZR]","[CHI_A12, COV3_A15, LAVES_C14, LAVES_C15, MU_P...",21.744828,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
737,1235,1976,"[0.10189047, 0.27404854, 0.3013579, 0.05023442...","[CO, CR, MO, NI, TA, V, W, ZR]",[LIQUID],22.094840,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325
738,1236,652,"[0.21985678, 0.18890162, 0.08386024, 0.0733089...","[CO, CR, MO, NI, TA, V, W, ZR]","[LAVES_C14, LAVES_C15, MU_PHASE, NI10ZR7, NITA...",22.048363,"[AL3M_D022, ALCR2_C11B, ALM_D019, ALTA_SIGMA, ...",101325


In [22]:

def getFullGM(db, components, all_phases,possible_phases, amounts, T, P = 101325):
    results = [np.nan] * len(all_phases)
    for i in range(len(all_phases)):
        if all_phases[i] in possible_phases:
            try:
                values = calculate(db, components, all_phases[i] , output = 'GM', P = P, T = T, points = amounts)
                results[i] = values.GM.values.flat[0]
            except Exception as e:
                print(len(calculate(db, components, all_phases[i], output = 'GM', P = P, T = T, pdens =1).GM))
                continue
    return results

def calcDFLine(df, ind, phase, db = db):
    temp = df.loc[ind, 'Temp']
    amounts = df.loc[ind, 'amounts']
    possible_phases = df.loc[ind, 'possible_phases']
    components = df.loc[ind, 'Components']
    components.append('VA')
    # return getFullGM(db, components,all_phases, possible_phases, amounts, T = temp)
    return calculate(db, components, phase, output = 'GM', P = P, T = temp, points = amounts)
# calcDFLine(df, 736, 'LAVES_C14').GM
calcDFLine(df, 736, 'CHI_A12').GM

<xarray.DataArray 'GM' (N: 1, P: 1, T: 1, points: 1)>
array([[[[-9794.64873038]]]])
Coordinates:
  * N        (N) float64 1.0
  * P        (P) float64 1.013e+05
  * T        (T) float64 566.0
Dimensions without coordinates: points

In [ ]:
i = df_origin.index[0]

for i in tqdm(df_origin.index):
    if i < 69:
        continue
    df_line = df_origin.loc[i:i].copy()
    df_line.loc[:,'gm_temp'] = df.loc[i:i].apply(lambda x: getTimeGM(db, x.Components, all_phases, x.possible_phases, x.amounts, x.Temp), axis=1)
    df_line.loc[:,'gm_time'] = df_line.apply(lambda x: x.gm_temp[0], axis=1)
    df_line.loc[:,'GM'] = df_line.apply(lambda x: x.gm_temp[1], axis=1)
    df_line = df_line.drop(['gm_temp'], axis=1)
    addDFLineToCSV(df_line)
    break

  9%|▉         | 69/740 [00:48<07:53,  1.42it/s]


In [ ]:
np.array(df_line['GM'][i])[~np.isnan(df_line['GM'][i])]

array([-31867.29331493, -24680.4982195 , -32080.37870313, -42951.99293437,
       -45575.09703745, -41100.34538737, -93451.1801303 , -24075.4628809 ,
       -25732.33635904, -27973.50522959])